In [125]:
import glob
from pathlib import Path

In [126]:

male_directory = "C:/Users/media/Desktop/gobbykid/corpus/gobbykidCorpus/male-writers"
female_directory = "C:/Users/media/Desktop/gobbykid/corpus/gobbykidCorpus/female-writers"

male_files = glob.glob(f"{male_directory}/*.txt")
female_files = glob.glob(f"{female_directory}/*.txt")

In order to better visualize the corpus metadata, we create three distint lists of dictionaries and as many .csv files.

In [127]:
import re

male_authors_data = []
female_authors_data = []
whole_original_corpus = []


for file in male_files:
    text = open(file, 'rb').read()
    book_dict = dict()

    book_dict['book_title'] = Path(file).stem
    book_dict['author'] = re.search("(?<=_)(.*?)(?=-)", book_dict['book_title']).group(0)
    book_dict['year'] = int(re.search("\d{3}.", book_dict['book_title']).group(0))
    book_dict['decade'] = (re.search("\d{3}", book_dict['book_title']).group(0)) + "0s"
    book_dict['tokens_in_book'] = len(text.split())
    book_dict['authors_sex'] = 'M'
    male_authors_data.append(book_dict)

for file in female_files:
    text = open(file, 'rb').read()
    book_dict = dict()

    book_dict['book_title'] = Path(file).stem
    book_dict['author'] = re.search("(?<=_)(.*?)(?=-)", book_dict['book_title']).group(0)
    book_dict['year'] = int(re.search("\d{3}.", book_dict['book_title']).group(0))
    book_dict['decade'] = (re.search("\d{3}", book_dict['book_title']).group(0)) + "0s"
    book_dict['tokens_in_book'] = len(text.split())
    book_dict['authors_sex'] = 'F'
    female_authors_data.append(book_dict)

whole_original_corpus = sorted((male_authors_data + female_authors_data), key=lambda dict: dict['book_title'])

In [128]:
#create a csv with metadata of non-balanced corpus, containing also the number of tokens per book

import csv
with open('m_data_table.csv', 'w', encoding='utf-8') as file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    for row in male_authors_data:
        writer.writerow(row)

with open('f_data_table.csv', 'w', encoding='utf-8') as file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    for row in female_authors_data:
        writer.writerow(row)

with open('whole_original_corpus.csv', 'w', encoding='utf-8') as file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    for row in whole_original_corpus:
        writer.writerow(row)

In [129]:
print("♂️ MALE AUTHORS CORPUS\nTotal number of words:",sum([row['tokens_in_book'] for row in male_authors_data]), "\nNumber of books:", len(male_files))
print("\n♀️ FEMALE AUTHORS CORPUS\nTotal number of words:",sum([row['tokens_in_book'] for row in female_authors_data]), "\nNumber of books:", len(female_files))


♂️ MALE AUTHORS CORPUS
Total number of words: 14074486 
Number of books: 161

♀️ FEMALE AUTHORS CORPUS
Total number of words: 9458669 
Number of books: 185


Criteri da tenere in considerazione per il bilanciamento:
1. numero di tokens maschili/femminili
2. Fase temporale (decennio?)
3. autore per decennio

L'output che vogliamo sono **due liste di documenti**, una per autori e una per autrici, che contengano il *massimo* numero di documenti per ogni decennio, con il *minimo* numero di libri dello stesso autore, con la *minima* differenza di numero di token tra maschi e femmine per ogni singolo decennio. La gerarchia di priorità è la seguente:

1. minima differenza assoluta di tokens maschi/femmine per decennio
2. massimo numero di libri per decennio
3. minimo numero di libri dello stesso autore per decennio

~~Considera la possibilità di usare il built-in `filter()` ed eventualmente importare e usare `random` (per scegliere documenti a caso una volta applicati i filtri condizionali).~~ 
~~Prima definisci le funzioni per verificare le condizioni dei filtri (ad esempio:  `return true if abs(male_document_tokens_number - female_document_tokens_number) < 10000`) e poi applica i filtri al dataset.~~

Ho deciso di bilanciare con i criteri spiegati qui di seguito. 
Dal momento che i libri delle autrici sono più numerosi ma generalmente più corti, mentre quelli degli autori sono di meno ma più lunghi, nel corpus finale per gli autori vanno inseriti quanti più libri possibile (che è un requirement che applichiamo anche al corpus femminile), però il più corti possibile; vice versa, per le autrici vogliamo mantenere sì quanti più libri possibile, ma devono essere i più lunghi tra quelli disponibili. Per mantenere un bilanciamento a livello di varietà di autore, e per farlo all'interno di ciascun decennio, ho suddiviso il corpus per decenni e ho applicato questo criterio di scelta in ciascun decennio *e ai libri di ciascun autore*. In poche parole, per il sottoinsieme di libri di ciascun decennio:
* il numero di autori presenti per decade è costante (tutti gli autori appaiono con *almeno un libro* in ciascun decennio). in tal modo si rappresenta anche la proporzione di numero di libri scritti da ciascun autore presente nel corpus. 
* la dimensione dei testi per ciascun autore, relativamente a ciascun decennio, è minimizzata per gli autori e massimizzata per le autrici. 
* c'è un solo parametro che è diverso tra corpus maschile e femminile, ovvero il numero n di testi da scegliere, rispettivamente tra i più corti e tra i più lunghi, per ciascun autore, relativamente a ciascun decennio. Con n=6 per il corpus maschile e n=10 per quello femminile, la differenza di token totali tra i due corpus di riduce a meno di 300'000, e allo stesso tempo garantisce un numero ampio di documenti. Possiamo regolare questo parametro a seconda delle nostre esigenze.




From the list `whole_original_corpus`, containing dictionaries storing the metadata for every document in both the corpora, i.e. male and female authors, we create a DataFrame object, `corpus_df`, in order to be able to use it for visualizations with Plotly. 

In [130]:
import pandas as pd

# transform the list into a DataFrame, for using it with Plotly 
corpus_df = pd.DataFrame(whole_original_corpus)

In [131]:
import plotly.express as px

whole_corpus_viz = px.scatter(
    corpus_df, 
    x= 'year',
    y='tokens_in_book',
    hover_name='book_title', 
    size='tokens_in_book', 
    marginal_x='histogram',
    trendline='ols',
    color='authors_sex', 
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
    title='The original (non balanced) corpus',
    labels={'year':'Year', 'tokens_in_book':'Number of tokens', 'authors_sex':"Author's sex"})


whole_corpus_viz.show()


The easiest way we found to tackle our problem was to filter 

IMPORTANTE: https://stackoverflow.com/questions/57597433/filtering-a-list-of-dictionaries-based-on-multiple-values

In [177]:
###--------------------------------------COPIA DI PROVA PER FUNZIONE UNICA!!!!!!!!!!!!!!!!-------------------------------

from itertools import groupby

def get_balanced_data(original_data: list, n_books_male=6, n_books_female=10) -> list:
    """
    Takes n books by each author for each decade. Takes the shortest books if corpus is male authors', the largest if it is females'. 
    Input: 
        original_data: list of dictionaries with the data of the corpus (male authors, female authors, or whole authors);
        n: number of books to include by each author for each decade: shortest books if male corpus, longest if female;
    Return: balanced data for the input corpus
    """
    auth_sex = ""
    balanced_data = []
    
    for key, group in groupby(sorted(original_data, key = lambda d: d['authors_sex']), key = lambda d: d['authors_sex']): #groups input data by authors_sex
        auth_list = list(group) #create one list for each group (by authors_sex)
        if auth_list[0]['authors_sex'] == 'M': #set parameters for male data
            auth_sex = 'M'
            n = n_books_male
        elif auth_list[0]['authors_sex'] == 'F': #set parameters for female data
            auth_sex = 'F'
            n = n_books_female
        
        for k,g in groupby(sorted(original_data, key = lambda d: d['decade']), key = lambda d: d['decade']): #groups the dicts by decade from a list sorted by decade
            l = list(g)
            sorted_list = sorted(l, key=lambda dict: dict['author'])
            to_add = []
            if auth_sex == 'M':
                to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[:n] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n FIRST elements from the list sorted by author
            
            elif auth_sex == 'F':
                to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[-n:] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n LAST elements from the list sorted by author
            
            balanced_data.extend(to_add)

    balanced_data = [dic for l in balanced_data for dic in l] #unpacks the list of lists of dictionaries into a list of dictionaries

    return balanced_data





In [179]:
male_balanced_data = get_balanced_data(male_authors_data)
female_balanced_data = get_balanced_data(female_authors_data)
balanced_data = get_balanced_data(whole_original_corpus, 6, 10)

print("New number of books in MALE authors corpus:", len(male_balanced_data))
print("New total number of tokens in MALE authors corpus:", sum([dictionary['tokens_in_book'] for dictionary in male_balanced_data]))

print("New number of books in FEMALE authors corpus:", len(female_balanced_data))
print("New total number of tokens in FEMALE authors corpus:", sum([dictionary['tokens_in_book'] for dictionary in female_balanced_data]))

print("New number of books in the WHOLE corpus:", len(balanced_data))
print("New total number of tokens in WHOLE corpus:", sum([dictionary['tokens_in_book'] for dictionary in balanced_data]))



New number of books in MALE authors corpus: 109
New total number of tokens in MALE authors corpus: 8797218
New number of books in FEMALE authors corpus: 157
New total number of tokens in FEMALE authors corpus: 8534374
New number of books in the WHOLE corpus: 524
New total number of tokens in WHOLE corpus: 35295125


In [134]:
from itertools import groupby

male_balanced_data = []

#TAKE THE N SHORTEST BOOKS FOR AUTHOR FOR DECADE
for k,g in groupby(sorted(male_authors_data, key = lambda d: d['decade']), key = lambda d: d['decade']): #groups the dicts by decade from a list sorted by decade
    l = list(g)
    sorted_list = sorted(l, key=lambda dict: dict['author'])
    n = 6 #number of SHORTEST book per author per decade to be included
    to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[:n] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n LAST elements from the sorted list
    male_balanced_data.extend(to_add)

male_balanced_data = [dic for l in male_balanced_data for dic in l] #unpacks the list of lists of dictionaries into a list of dictionaries
print("New number of books in male corpus:", len(male_balanced_data))
print("New total number of tokens in male corpus:", sum([dictionary['tokens_in_book'] for dictionary in male_balanced_data]))



New number of books in male corpus: 109
New total number of tokens in male corpus: 8797218


In [135]:
from itertools import groupby


female_balanced_data = []

#TAKE THE N SHORTEST BOOKS FOR AUTHOR FOR DECADE
for k,g in groupby(sorted(female_authors_data, key = lambda d: d['decade']), key = lambda d: d['decade']): #groups the dicts by decade from a list sorted by decade
    l = list(g)
    sorted_list = sorted(l, key=lambda dict: dict['author'])
    n = 10 #number of LARGEST book per author per decade to be included
    to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[-n:] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n LAST elements from the sorted list
    female_balanced_data.extend(to_add)


female_balanced_data = [dic for l in female_balanced_data for dic in l] #unpacks the list of lists of dictionaries into a list of dictionaries
print("New number of books in female corpus:", len(female_balanced_data))
print("New total number of tokens in female corpus:", sum([dictionary['tokens_in_book'] for dictionary in female_balanced_data]))

New number of books in female corpus: 157
New total number of tokens in female corpus: 8534374


In [136]:
whole_balanced_data = sorted((male_balanced_data + female_balanced_data), key=lambda dict: dict['book_title'])

In [137]:
#save balanced corpus metadata in a .csv file
with open('m_balanced_corpus.csv', 'w', encoding='utf-8') as output_file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(output_file, fieldnames=fields)
    writer.writeheader()
    for row in male_balanced_data:
        writer.writerow(row)

with open('f_balanced_corpus.csv', 'w', encoding='utf-8') as output_file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(output_file, fieldnames=fields)
    writer.writeheader()
    for row in female_balanced_data:
        writer.writerow(row)

with open('whole_balanced_corpus.csv', 'w', encoding='utf-8') as output_file:
    fields = ['book_title','author','year','decade','tokens_in_book', 'authors_sex']
    writer = csv.DictWriter(output_file, fieldnames=fields)
    writer.writeheader()
    for row in whole_balanced_data:
        writer.writerow(row) 

This last step copies the files from the old folder and paste them into the folder of the new, balanced, corpus. 

In [138]:
import shutil
from pathlib import Path 

male_balanced_corpus = sorted([dict['book_title'] for dict in male_balanced_data])
female_balanced_corpus = sorted([dict['book_title'] for dict in female_balanced_data])

m_balanced_corpus_path = "C:/Users/media/Desktop/gobbykid/balanced_corpus/m/"
f_balanced_corpus_path = "C:/Users/media/Desktop/gobbykid/balanced_corpus/f/"


for file in male_files:
    filename = Path(file).stem
    if filename in male_balanced_corpus:
        shutil.copy(file, m_balanced_corpus_path+filename+".txt")

for file in female_files:
    filename = Path(file).stem
    if filename in female_balanced_corpus:
        shutil.copy(file, f_balanced_corpus_path+filename+".txt")

In [139]:
balanced_corpus_df = pd.DataFrame(whole_balanced_data)

In [140]:
balanced_corpus_viz = px.scatter(
    balanced_corpus_df, 
    x= 'year',
    y='tokens_in_book',
    hover_name='book_title', 
    size='tokens_in_book', 
    marginal_x='histogram',
    trendline='ols',
    color='authors_sex', 
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
    title='The balanced corpus',
    labels={'year':'Year', 'tokens_in_book':'Number of tokens', 'authors_sex':"Author's sex"})


balanced_corpus_viz.show()